In [1]:
Sys.setlocale(category = "LC_ALL", locale = "en_US.UTF-8")

[1] "en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/C"

# Dependencies

# Functions

In [2]:
source("/Users/anabbi/git/ped_CapTCRseq/R/Immune_Diversity.R")

# Paths

In [3]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

# Main

In [4]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/"

## TRB

In [7]:
file_list <- list.files(h4hpath, pattern = paste0("CLONES_", "TRB"), recursive = T)

In [10]:
tail(file_list)

[1] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_365-03_cfDNA.txt"
[2] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_370-02_cfDNA.txt"
[3] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_374-03_cfDNA.txt"
[4] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_378-01_cfDNA.txt"
[5] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_381-04_cfDNA.txt"
[6] "remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_383-03_cfDNA.txt"

Some files are duplicated. Make a table to identify and pick the larger clone file

In [11]:
file_df <- as.data.frame(file_list)

In [12]:
file_df$samplename <- gsub(".*CLONES_TRB", "", file_df$file_list)
file_df$samplename <- gsub(".txt", "", file_df$samplename)

In [13]:
file_df$size <- NA
for (i in 1:nrow(file_df)) {
    file_df$size[i] <- file.size(paste0(h4hpath, file_df$file_list[i]))
}


In [14]:
head(file_df)

,file_list,samplename,size
,<chr>,<chr>,<dbl>
1,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA,14759
2,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt,CHP_350-01-PBMC-DNA,85631
3,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-02-PBMC-DNA.txt,CHP_350-02-PBMC-DNA,143632
4,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-03-PBMC-DNA.txt,CHP_350-03-PBMC-DNA,107190
5,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-04-PBMC-DNA.txt,CHP_350-04-PBMC-DNA,54188
6,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_353-02-PBMC-DNA.txt,CHP_353-02-PBMC-DNA,9062


In [15]:
file_df[ grepl("383-03", file_df$file_list),]

,file_list,samplename,size
,<chr>,<chr>,<dbl>
380,batch8_14Jan2020/mixcr/clones/CLONES_TRBCHP_383-03-PBMC-DNA.txt,CHP_383-03-PBMC-DNA,1469
780,immunarch/cloneslinks/CLONES_TRBCHP_383-03_cfDNA.txt,CHP_383-03_cfDNA,NA
781,immunarch/cloneslinks/CLONES_TRBCHP_383-03-PBMC-DNA.txt,CHP_383-03-PBMC-DNA,NA
1223,remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_383-03_cfDNA.txt,CHP_383-03_cfDNA,1998


In [16]:
# remove immunarch files
file_df <- file_df[ !grepl("immunarch", file_df$file_list),]

In [17]:
#if duplicated keep the bigger file corresponding to deeper seqeuncing.
file_df_dedup <- file_df[0,]

for(s in unique(file_df$samplename)){
    tmp <- file_df[ file_df$samplename == s,]
    if(nrow(tmp) ==1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp)
    }
    if(nrow(tmp) > 1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp[ tmp$size == max(tmp$size),])
    }    
}

In [18]:
file_df_dedup[ grepl("343-05", file_df_dedup$file_list),]

,file_list,samplename,size
,<chr>,<chr>,<dbl>
208,batch2_Aug62019/mixcr/clones/CLONES_TRBCHP_343-05-PBMC-DNA.txt,CHP_343-05-PBMC-DNA,28197
1164,remaining_batch_3_cfDNA_15April2021/mixcr/clones/CLONES_TRBCHP_343-05-cfDNA.txt,CHP_343-05-cfDNA,2652


In [19]:
dim(file_df)
dim(file_df_dedup)

[1] 620   3

[1] 605   3

In [20]:
file_df_dedup_trb <- file_df_dedup
save(file_df_dedup_trb, file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [ ]:
batchTRB <- immunelistfx(file_df_dedup$file_list, h4hpath, "TRB")

In [ ]:
names(batchTRB) <- gsub(".*CLONES_TRB", "", names(batchTRB))

In [ ]:
head(names(batchTRB))

In [ ]:
TRBlist <- batchTRB 

In [ ]:
save(TRBlist, file = paste0(datapath,"capTCRseq/TRBlist.RData"))

### divstats

In [19]:
load(file = paste0(datapath,"capTCRseq/TRBlist.RData"))
load(file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [10]:
TRBlist <- TRBlist[sapply(TRBlist, function(x) length((x)) != 0)]
length(names(TRBlist))

[1] 580

In [20]:
TRBlist[64:65]

$`CHP_358-02_cfDNA`
 [1] 1745 1723 1493 1238  930  775  765  728  697  455  433  212  185  110   33
[16]    1

$`CHP_361-02_cfDNA`
[1] 1318  700  649  333   61   40

In [11]:
# add filenames to divstats
myfilenames <- file_df_dedup_trb[ file_df_dedup_trb$samplename %in% names(TRBlist),]

In [12]:
head(myfilenames)

,file_list,samplename,size
,<chr>,<chr>,<dbl>
264,batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA,16011
2,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-01-PBMC-DNA.txt,CHP_350-01-PBMC-DNA,85631
3,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-02-PBMC-DNA.txt,CHP_350-02-PBMC-DNA,143632
4,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-03-PBMC-DNA.txt,CHP_350-03-PBMC-DNA,107190
5,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_350-04-PBMC-DNA.txt,CHP_350-04-PBMC-DNA,54188
6,batch_reseq_24Feb2020/mixcr/clones/CLONES_TRBCHP_353-02-PBMC-DNA.txt,CHP_353-02-PBMC-DNA,9062


In [21]:
Divstats.fx(TRBlist, "TRB", "CHP", paste0(datapath, "capTCRseq/"))

In [ ]:
# save(TRBlist, file = "/Users/anabbi/Desktop/H4H/INTERCEPT/analysis/metadata/TRBlist.RData")
# save(file_df_dedup_trb, file = "/Users/anabbi/Desktop/H4H/INTERCEPT/analysis/metadata/TRBfiles.RData")

### Cleanup divstats

In [24]:
divstats <- read.csv(paste0(datapath, "capTCRseq/divstats_TRBCHP.csv"), header = T, stringsAsFactors = F, row.names = 1)

In [26]:
divstats["CHP_358-02_cfDNA",]

,TRB,Reads,CPKR,Average_reads,VMR,Max_reads,Singletons,Doubletons,qD,Sample_Coverage,observed_Richness,estimated_Richness,SE_Richeness,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CHP_358-02_cfDNA,16,11523,1.388527,720.1875,0.8040794,1745,1,0,16,1,16,16,0.482,11.428,11.436,0.064,9.962,9.97,0.073


In [27]:
# add 9 samples with nonproductive cdr3
addsamples <- c(
    "CHP_347-01-cfDNA", "CHP_366-05_cfDNA", "CHP_377-01-cfDNA",
    "CHP_381-05_cfDNA", "CHP_391-02_cfDNA", "CHP_392-01_cfDNA",
    "CHP_398-01_cfDNA", "CHP_401-03_cfDNA", "CHP_419-FFPE-DNA_S2"
)


In [28]:
divstats[ nrow(divstats) + 9 , ] <- NA

In [29]:
rownames(divstats)[is.na(divstats$TRB)] <- addsamples

In [30]:
#example sample_id CHP_xxx_xx
divstats$sample_id <- rownames(divstats)

#example basename CHP_xxx-xx-PBMC-DNA
divstats$basename <- rownames(divstats)

In [31]:
# add filenames to divstats
divstats$file <- myfilenames$file_list[ match(divstats$sample_id, myfilenames$samplename)]

Separate samples

In [32]:
#divstats$Sample <- NULL

In [33]:
divstats$Sample <- NA
divstats$Sample[grepl("PBMC", rownames(divstats))] <- "PBMC"
divstats$Sample[grepl("cfDNA", rownames(divstats))] <- "cfDNA"
divstats$Sample[grepl("FFPE", rownames(divstats))] <- "Tumor"

Separate LFS and non-LFS

In [34]:
divstats$Group <- NA
divstats$Group[grepl("LFS", rownames(divstats))] <- "LFS"
divstats$Group[grepl("CHP", rownames(divstats))] <- "CHP"

In [37]:
summary(divstats$Reads)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
     1.0    841.8   3088.5  18710.4  12587.8 363060.0        9 

Get patient ids

In [38]:
divstats$Patient <- NA
# CHP
divstats$Patient[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],1,7)
# Normal samples have either YO or MO
divstats$Patient[grepl("YO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("YO", rownames(divstats))])
divstats$Patient[grepl("MO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("MO", rownames(divstats))])
# LFS
divstats$Patient[divstats$Group == "LFS"] <- substr(rownames(divstats)[divstats$Group == "LFS"],1,12)

Get cycles. For LFS, Normals and Tumors recode to 00

In [39]:
divstats$Cycle <- NA
divstats$Cycle[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],9,10)
divstats$Cycle[grepl("YO", rownames(divstats))] <- "00"
divstats$Cycle[grepl("MO", rownames(divstats))] <- "00"
divstats$Cycle[ divstats$Sample == "Tumor"] <- "00"
divstats$Cycle[ divstats$Group == "LFS"] <- "00"

In [40]:
divstats$sample_id <- NA

# bind patient and cycle for PBMC and cfdna to generate sample id
divstats$sample_id[divstats$Sample != "Tumor"] <- paste(divstats$Patient[divstats$Sample != "Tumor"], "_",
    divstats$Cycle[divstats$Sample != "Tumor"],
    sep = ""
)

# For tumor samples start with rownames because some of them are two biopsies from the same tumor
divstats$sample_id[divstats$Sample == "Tumor"] <- gsub("-FFPE-DN.*", "", rownames(divstats)[divstats$Sample == "Tumor"])


In [41]:
divstats$sample_id <- gsub("-", "_", divstats$sample_id)

In [42]:
divstats$Patient <- NULL
divstats$Cycle <- NULL
divstats$Sample <- NULL
divstats$Group <- NULL

In [43]:
tail(divstats,10)

,TRB,Reads,CPKR,Average_reads,VMR,Max_reads,Singletons,Doubletons,qD,Sample_Coverage,...,SE_Richeness,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,sample_id,basename,file
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
CHP_383-03_cfDNA,2,1135,1.762115,567.5,1.117665,1016,0,0,2,1,...,0,1.399,1.399,0.028,1.231,1.231,0.02,CHP_383_03,CHP_383-03_cfDNA,remaining_batch_5_cfDNA_22April2021/mixcr/clones/CLONES_TRBCHP_383-03_cfDNA.txt
CHP_347-01-cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,CHP_347_01,CHP_347-01-cfDNA,NA
CHP_366-05_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,CHP_366_05,CHP_366-05_cfDNA,NA
CHP_377-01-cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,CHP_377_01,CHP_377-01-cfDNA,NA
CHP_381-05_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,CHP_381_05,CHP_381-05_cfDNA,NA
CHP_391-02_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,CHP_391_02,CHP_391-02_cfDNA,NA
CHP_392-01_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,CHP_392_01,CHP_392-01_cfDNA,NA
CHP_398-01_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,CHP_398_01,CHP_398-01_cfDNA,NA
CHP_401-03_cfDNA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,CHP_401_03,CHP_401-03_cfDNA,NA


In [44]:
write.csv(divstats,
          paste0(datapath, "capTCRseq/divstats_TRBCHP.csv"), row.names = TRUE)